### Library

In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_file, show
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook
from bokeh.transform import cumsum
from bokeh.palettes import PuBu
from math import pi
from bokeh.palettes import YlGnBu
from bokeh.palettes import Blues
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, IndexFilter
from pyod.models.mad import MAD

## Dataset

In [2]:
origin_data = 'data_prueba_tecnica.csv'

In [3]:
def read_dataset(origin_data):
  """Read a dataset.

  Read a dataset from the Raw folder.

   Args:
      origin_data: Path of where the data set is stored.

  Returns:
        Return a data set with the help of the pandas library.
  """

  df_dataset = pd.read_csv(origin_data)
  return(df_dataset)

In [4]:
df_dataset = read_dataset(origin_data)

In [5]:
df_dataset.shape

(10000, 7)

## 1.Category classification

In [6]:
def variables_classifier(df_dataset, porc_nulos=.5, porc_lim_unitario=.75,
                         porc_unique_text=.20, porc_unique_num=.01):
    """ Field analysis description.

    Analyze by field the number of nulls, type of data it contains, unique values,
    maximum frequency to know which category the field corresponds to.

    Args:
        df_dataset: Data set being analyzed.
        perc_null: Limit to classify a 'Nula' category field.
        perc_lim_unitary: Limit to classify a 'Unitaria' category field.
        perc_unique_text: Limit to classify a 'Discreta' category field.
        perc_unique_num: Limit to classify a 'Discreta' category field.

    Returns:
         A dataframe with the category that corresponds to each field.
         example:
         if perc_null >= .5 then category field is 'Nula'
         if perc_lim_unitary >= .75 then category field is 'Unitaria'
         if perc_unique_text < .20 then category field is 'Discreta'
         if perc_unique_num < .01 then category field is 'Discreta'
    """

    variables = df_dataset.columns.to_list()
    df_columnas = pd.DataFrame(variables, columns=['Categoría'])
    df_columnas['Type'] = df_columnas['Categoría'].map(df_dataset.dtypes)
    df_columnas['Count'] = df_columnas['Categoría'].map(df_dataset.count())
    df_columnas['Nulos'] = df_columnas['Categoría'].map(df_dataset.shape[0]-df_dataset.count())
    df_columnas['Porc_nulos'] = df_columnas['Nulos']/df_dataset.shape[0]
    df_columnas['Unique'] = df_columnas['Categoría'].map(df_dataset.nunique())
    df_columnas['Porc_unique'] = df_columnas['Unique']/df_dataset.shape[0]

    maxi = []
    for i in variables:
        maxi.append(df_dataset[i].value_counts().to_list()[0])
    df_columnas['Frec_max'] = maxi
    df_columnas['Porc_frec_max'] = df_columnas['Frec_max'] / df_dataset.shape[0]

    #Clasificación
    tipos = []
    for i in range(df_columnas.shape[0]):
        if df_columnas.loc[i]['Porc_nulos'] >= porc_nulos:
            tipos.append('Nula')
        elif df_columnas.loc[i]['Porc_frec_max'] >= porc_lim_unitario:
            tipos.append('Unitaria')
        elif df_columnas.loc[i]['Type'] != 'int64' and df_columnas.loc[i]['Type'] != 'float64':
            if df_columnas.loc[i]['Porc_unique'] < porc_unique_text:
                tipos.append('Discreta')
            else:
                tipos.append('Texto')
        elif df_columnas.loc[i]['Porc_unique'] < porc_unique_num:
            tipos.append('Discreta')
        else:
            tipos.append('Continua')
    df_columnas['Tipo'] = tipos
    return(df_columnas)

In [7]:
df_columnas = variables_classifier(df_dataset)

In [8]:
vard = df_columnas[df_columnas['Tipo'] == 'Discreta']['Categoría'].to_list()

In [9]:
varc = df_columnas[df_columnas['Tipo'] == 'Continua']['Categoría'].to_list()

### Discrete Field Graphs

In [10]:
def graf_disc1(discrete_field):
    """ Bar graphic.

    Generates a bar graph of the discrete variables, in this case of the 'status' field.

    Args:
      discrete_field: List of fields containing discrete variables

    Returns:
         A Bar graphic with information from the 'status' column.
    """
    datos = df_dataset[discrete_field].value_counts().index.tolist()
    counts = df_dataset[discrete_field].value_counts()

    source = ColumnDataSource(data=dict(datos=datos, counts=counts))

    p = figure(x_range=datos, height=350, toolbar_location=None, title=str(discrete_field))
    p.vbar(x='datos', top='counts', width=0.8, source=source, legend_field="datos",
           line_color='white', fill_color=factor_cmap('datos', palette=PuBu[9], factors=datos))

    p.xgrid.grid_line_color = None
    p.y_range.start = 0
    p.y_range.end = 15
    p.legend.orientation = "horizontal"
    p.legend.location = "top_center"

    output_notebook()
    return show(p)

In [11]:
graf_disc1(vard[0])

###Continuous Field Graphs

In [12]:
def graf_cont1(continuous_field):
    """ Graphic to identify outliers.

    Generate a graph to identify the outliers of the continuous variables,
    in this case of the 'amount' field.

    Args:
      continuous_field: List of fields containing continuous variables.

    Returns:
         A Graphic with information from the 'amount' column.
    """
    source = ColumnDataSource(data=dict(x= df_dataset[continuous_field].value_counts().index.tolist(), y= df_dataset[continuous_field].value_counts()))
    tools = ["box_select", "hover", "reset"]
    p = figure(title  = str(continuous_field), height=300, width=300, tools=tools)
    p.segment(0, df_dataset[continuous_field].value_counts(), df_dataset[continuous_field].value_counts().index.tolist(), df_dataset[continuous_field].value_counts(), line_width=2, line_color="blue")
    p.circle(df_dataset[continuous_field].value_counts().index.tolist(), df_dataset[continuous_field].value_counts(), size=15, fill_color="skyblue", line_color="blue", line_width=3, )
    output_notebook()
    return show(gridplot([[p]]))


In [13]:
graf_cont1(varc)

In [14]:
def graf_cont2(continuous_field):
    """ Graphic to identify outliers.

    Generate a graph to identify the outliers of the continuous variables,
    in this case of the 'amount' field.

    Args:
      continuous_field: List of fields containing continuous variables.

    Returns:
         A Graphic with information from the 'amount' column.
    """
    source = ColumnDataSource(data=dict(x= df_dataset[continuous_field].value_counts().index.tolist(),
                                        y= df_dataset[continuous_field].value_counts()))
    view = CDSView(source=source, filters=[IndexFilter([0, 1, 2])])

    tools = ["box_select", "hover", "reset"]
    p = figure(title  = str(continuous_field), height=300, width=300, tools=tools)
    p.circle(x="x", y="y", size=10, hover_color="red", source=source)
    p.line(df_dataset[continuous_field].value_counts().index.tolist(), df_dataset[continuous_field].value_counts(),
           line_width=2, line_color="skyblue")

    p_filtered = figure(height=300, width=300, tools=tools)
    p_filtered.circle(x="x", y="y", size=10, hover_color="steelblue",
                      source=source, view=view)
    output_notebook()
    return show(gridplot([[p, p_filtered]]))


In [15]:
graf_cont2(varc)

## 2.Missing

### Graphics Function

In [16]:
def valores_nulos(field):
    """ Graphic containing null value information.

    Generate a graph to identify the null values of the continuous or discrete variables.

    Args:
      field: List of fields containing continuous or discrete variables.

    Returns:
         A graph with information on the percentage of nulls for each field.
    """
    z = {'Nulos': df_dataset[field].isnull().sum(), 'No nulos': df_dataset[field].notnull().sum()}
    x = z
    data = pd.Series(x).reset_index(name='value').rename(columns={'index':'country'})
    data['angle'] = data['value']/data['value'].sum()*2*pi
    if len(x) < 3:
        data['color'] = ['#084594', '#2171b5']
    else:
        data['color'] =PuBu[len(x)]
    p = figure(height=350, title=str(field), toolbar_location=None,
               tools="hover", tooltips="@country: @value", x_range=(-0.5, 1.0))

    p.wedge(x=0, y=1, radius=0.4,
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
            line_color="white", fill_color='color', legend_field='country', source=data)

    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color = None
    output_notebook()
    return show(p)

### Discrete Field Graphs

In [17]:
for c in vard:
    valores_nulos(c)

### Continuous Field Graphs

In [18]:
for c in varc:
    valores_nulos(c)

### Null treatment

In [19]:
def Generar_valores_estadisticos(df_dataset, df_columnas):
    """ Statistical information for each field.

    Generates statistical information for each field, for example, mode, mean,
    std, median, maximum, minimum.

    Args:
      df_dataset: Dataframe that contains information about payments from different companies .
      df_columns: Dataframe containing the categories to which the fields belong

    Returns:
         A dataframe with the statistical information of each field.
    """
    df_columnas['Moda'] = df_columnas['Categoría'].map(df_dataset.mode().T[0])
    df_columnas['Media'] = df_columnas['Categoría'].map(df_dataset.mean())
    df_columnas['std'] = df_columnas['Categoría'].map(df_dataset.std())
    df_columnas['Mediana'] = df_columnas['Categoría'].map(df_dataset.median())
    df_columnas['Maximo'] = df_columnas['Categoría'].map(df_dataset.max())
    df_columnas['Minimo'] = df_columnas['Categoría'].map(df_dataset.min())
    df_describe_columnas_disc = df_columnas[(df_columnas['Nulos'] > 0) & (df_columnas['Tipo'] == 'Discreta')]
    return(df_describe_columnas_disc[['Categoría', 'Tipo', 'Nulos', 'Porc_nulos',
                                      'Moda', 'Frec_max', 'Porc_frec_max']])

In [20]:
df_disc = Generar_valores_estadisticos(df_dataset, df_columnas)

<ipython-input-19-8b26e3081ec7>:15: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_columnas['Media'] = df_columnas['Categoría'].map(df_dataset.mean())
<ipython-input-19-8b26e3081ec7>:16: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_columnas['std'] = df_columnas['Categoría'].map(df_dataset.std())
<ipython-input-19-8b26e3081ec7>:17: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select o

In [21]:
def trat_null_disc(df_dataset, discrete_field, tratamiento_disc):
    """ Null value treatment.

    Treatment of null values ​​for discrete variables.

    Args:
      df_dataset: Dataframe that contains information about payments from different companies .
      discrete_field: List of fields containing discrete variables
      tratamiento_disc: Method with which we are going to replace null values

    Returns:
         A dataframe with the null values ​​replaced by the mode.
         example:
         'Nan' = 	"pending_payment"
    """
    if tratamiento_disc == 'moda':
        #Completando valores faltantes con la moda datos categóricos
        for c in discrete_field:
            moda = df_dataset[c].mode()[0]
            df_dataset[c] = df_dataset[c].fillna(moda)
    return(df_dataset)

In [22]:
tratamiento_disc = "moda"

In [23]:
processing__nulls_dataset = trat_null_disc(df_dataset, vard, tratamiento_disc)

## 3.Outliers

In [24]:
def tabla_conjunta(df_dataset, df_columnas, porc_nulos=.5, porc_lim_unitario=.75,
                   porc_unique_text=.20, porc_unique_num=.01):
    """ Concatenate two dataframes.

    Concatenate the dataframes 'df_columns' and 'df_disc'.

    Args:
        df_dataset: Dataframe that contains information about payments from different companies .
        df_columns: Dataframe containing the categories to which the fields belong.
        perc_nulos: Limit to classify a 'Nula' category field.
        perc_lim_unitario: Limit to classify a 'Unitaria' category field.
        perc_unique_text: Limit to classify a 'Discreta' category field.
        perc_unique_num: Limit to classify a 'Discreta' category field.

    Returns:
         A  Dataframe containing the categories to which the fields belong and statistical information.
    """
    return pd.concat([df_columnas, df_disc], axis=0)

In [25]:
union_tables = tabla_conjunta(df_dataset, df_columnas, porc_nulos = .5, porc_lim_unitario = .75,
               porc_unique_text = .20, porc_unique_num = .01)

In [26]:
def outliers(df_dataset, field):
    """ Identify the number of outliers.

    Identifies the number of outlier values ​​of continuous variables.

    Args:
      df_dataset: Dataframe that contains information about payments from different companies .
      field: List of fields containing continuous or discrete variables.

    Returns:
         The number of outliers in the dataframe for the amount variable.
    """
    for c in field:
      mad = MAD(threshold=3.5)
      total_reshaped = df_dataset[field].values.reshape(-1, 1)
      labels = mad.fit(total_reshaped).labels_
      num_outliers = labels.sum()
    return(num_outliers)

In [27]:
num_outliers = outliers(df_dataset, varc)

In [28]:
def trat_outliers(df_dataset, field):
    """ Treatment of outliers.

   Treatment of outliers for continuous variables.

    Args:
      df_dataset: Dataframe that contains information about payments from different companies.
      field:  List of fields containing continuous variables.

    Returns:
         A dataframe with the reduction of outliers through the quantile method.
    """
    seventy_fifth = df_dataset[field].quantile(0.75)
    twenty_fifth = df_dataset[field].quantile(0.25)
    iqr = seventy_fifth - twenty_fifth
    lim_max = seventy_fifth + (1.5 * iqr)
    lim_min = twenty_fifth + (1.5 * iqr)

    for i in range(len(field)):
        df_dataset = df_dataset[(df_dataset[field[i]] > lim_min[i]) & (df_dataset[field[i]] < lim_max[i])]
    return(df_dataset)

##4.Aggregation Functions

In [29]:
dataset = trat_outliers(processing__nulls_dataset,varc)

In [30]:
dataset.shape

(866, 7)

In [31]:
dataset.head()

,id,name,company_id,amount,status,created_at,paid_at
4,6ccfc4c24e788e4bca448df343698782db6b0c0b,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,184.49,paid,2019-02-05,2019-02-05
18,798166c4cd9d464ab5e83b4152cfd191424bb34d,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,166.65,pending_payment,2019-05-10,2019-02-27
21,0dca80440f819c7f3077d534e92a9634d66e279a,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,163.49,paid,2019-04-03,2019-04-03
22,6a528f598c344398ce78afb995a2f52c82395f24,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,188.70,paid,2019-05-10,2019-05-10
25,be79778857c376c2d7c4500edcfa7eb00ab9ca47,MiPasajefy,cbf1c8b09cd5b549416d49d220a40cbd317f952e,158.33,paid,2019-04-11,2019-04-11


In [32]:
dataset['status'].unique()

array(['paid', 'pending_payment', 'voided', 'refunded', 'pre_authorized',
       'charged_back'], dtype=object)

In [33]:
dataset['name'].unique()

array(['MiPasajefy', 'Muebles chidos'], dtype=object)

In [73]:
def sales_analysis(dataset):
    """ Query in the dataset about sales.

    Queries in the dataset about daily sales by companies and their status.

    Args:
      dataset: Dataframe that contains information about payments from different companies.

    Returns:
         Ten Queries in the dataset about daily sales by companies and their status
    """
    #Daily Sales by company
    sales_day_company = dataset.groupby(['created_at', 'name']).agg({'name': ['count']})

    #Daily Sales by company and status
    sales_day_comp_status = dataset.groupby(['created_at', 'name', 'status']).agg({'name': ['count']})

    #Daily sales amount by company
    sales_amount = dataset.groupby(['created_at', 'name']).agg({'amount': ['sum']})

    #Maximum, minimum, average daily sales by company
    amount_functions = dataset.groupby(['created_at', 'name']).agg(max_amount=('amount', max),
                                            min_amount=('amount', min),
                                            mean_amount=('amount', np.mean))

    #Daily sales that do not contain voided sales
    sales_different_voided = dataset[dataset['status'] != 'voided'].groupby(['created_at', 'name']).agg({'name': ['count']})

    #Amount of daily sales canceled
    sales_canceled = dataset[dataset['status'] == 'voided'].groupby(['created_at', 'name']).agg({'amount': ['sum']})

    #Amount of canceled sales of the company 'MiPasajefy'
    amount_sales_can_pasajefy = dataset[(dataset['status'] == 'voided') & (dataset['name'] == 'MiPasajefy')].groupby(['created_at', 'name']).agg({'amount': ['sum']})

    #Amount of canceled sales of the company ''Muebles chidos"
    amount_sales_can_muebles = dataset[(dataset['status'] == 'voided') & (dataset['name'] != 'MiPasajefy')].groupby(['created_at', 'name']).agg({'amount': ['sum']})

    # Amount of paid sales of the company 'MiPasajefy'
    amount_paid_sales = dataset[(dataset['status'] == 'paid') & (dataset['name'] == 'MiPasajefy')].groupby(['created_at', 'name']).agg({'amount': ['sum']})

    #Amount of sales pending payment of the company 'MiPasajefy'
    amount_pending_sales = dataset[(dataset['status'] == 'pending_payment') & (dataset['name'] == 'Muebles chidos')].groupby(['created_at', 'name']).agg({'amount': ['sum']})

    return(sales_day_company, sales_day_comp_status, sales_amount, amount_functions,
           sales_different_voided, sales_canceled, amount_sales_can_pasajefy,
           amount_sales_can_muebles, amount_paid_sales, amount_pending_sales)

In [75]:
sales_day_company, sales_day_comp_status, sales_amount, amount_functions, sales_different_voided, sales_canceled, amount_sales_can_pasajefy,amount_sales_can_muebles, amount_paid_sales, amount_pending_sales = sales_analysis(dataset)